In [1]:
# the correlate functions in conv1d_prototype is too slow for neural network

# parallelization could be implemented on:
# - 1d level, by using numpy.dot, which is already highly optimized
# - 3d level (batch parallel), by multiprocessing libraries

In [2]:
import numpy as np

## 1d

In [5]:
def correlate1(X, K):
    """1d correlation"""
    
    xw = len(X)
    kw = len(K)
    ow = xw - kw + 1
    
    if ow < 1:
        return correlate1(K, X)
    
    # extract windows into ndarray
    windows = np.array([X[i:i+kw] for i in range(ow)])
    
    # perform dot product
    return np.dot(windows, K.T)

def convolve1(X, K):
    return correlate1(X, np.flip(K))

In [6]:
a = np.array([1,2,3,4,5,6,7,8])
b = np.array([1,0])

print(correlate1(a,b))
%timeit correlate1(a,b)

[1 2 3 4 5 6 7]
7.89 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [39]:
# old implementation
def correlate12(X, K):
    """1d correlation"""
    
    xw = len(X)
    kw = len(K)
    ow = len(X) - len(K) + 1
    
    if ow >= 1:
        return np.fromiter((np.sum(X[i:i+kw] * K) for i in range(ow)), dtype=float)
    else:
        return correlate1(K, X)

In [40]:
print(correlate12(a,b))
%timeit correlate12(a,b)
# 3-4x slower

[1. 2. 3. 4. 5. 6. 7.]
31.1 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
# scipy built-in
from scipy.signal import correlate
print(correlate(a, b, mode='valid'))
%timeit correlate(a, b, mode='valid')

In [89]:
# numpy built-in
print(np.correlate(a,b, mode='valid'))
%timeit np.correlate(a,b,mode='valid')

[1 2 3 4 5 6 7]
1.33 µs ± 47.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [28]:
# ok i've gave up implementing by myself
def correlate1(X, K, stride=1):

    ow = len(X) - len(K)
    assert ow % stride == 0, "Invalid stride"
    
    # somehow np.correlate is not commutative requiring manual fix
    if ow >= 0:
        return np.correlate(X, K, mode='valid')[::stride]
    else:
        return np.correlate(K, X, mode='valid')[::stride]   

def convolve1(X, K, stride=1):
    return correlate1(X, np.flip(K), stride=stride)

In [33]:
a = np.arange(10)
print(a)
b = np.arange(5)
print(b)

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4]


In [38]:
print(correlate1(a, b, stride=1))
print(correlate1(a, b, stride=5))

[30 40 50 60 70 80]
[30 80]


In [11]:
np.array([1,2,3,4,5,6,7])[::2]

array([1, 3, 5, 7])

## 2d

In [42]:
def correlate2(xs, ks, stride=1):
    """1d correlation"""

    ow = xs.shape[1] - ks.shape[1]
    assert ow % stride == 0, "Invalid stride"
    
    if ow < 0:
        return correlate2(ks, xs, stride=stride)
    
    # divide into 1d conrrelation
    out = np.array([np.correlate(x, k, mode='valid')[::stride] for x, k in zip(xs, ks)])
    
    # sum into 1d output
    return np.sum(out, axis=0)
    
def convolve2(xs, ks, stride=1):
    return correlate2(xs, np.flip(ks, axis=1), stride=stride)

In [43]:
a = np.arange(100).reshape(4,25)
print(a)
b = np.arange(20).reshape(4,5)
print(b)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24]
 [25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
  49]
 [50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
  74]
 [75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98
  99]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


In [48]:
print(correlate2(a, b, stride=5))

[10670 11620 12570 13520 14470]


In [52]:
def correlate3(xss, kss, stride=1):
    xshape = xss.shape
    kshape = kss.shape
    
    assert (len(xshape)==3 and len(kshape)==3), "3d input required"
    assert xshape[1] == kshape[1], "kernels not spanning input height"
    
    return np.array([[correlate2(xs, ks, stride=stride) for ks in kss] for xs in xss])

def convolve3(xss, kss, stride=1):
    return correlate3(xss, np.flip(xss, axis=2), stride=stride)

In [55]:
a = np.arange(16).reshape(2,2,4)
print(a)
b = np.arange(8).reshape(2,2,2)
print(b)

[[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]]
[[[0 1]
  [2 3]]

 [[4 5]
  [6 7]]]


In [57]:
correlate3(a, b, stride=2)

array([[[ 24,  36],
        [ 64, 108]],

       [[ 72,  84],
        [240, 284]]])